In [1]:
import pandas as pd
import sqlite3
import random
from sqlite3 import Error
import hashlib

In [2]:
sales = pd.read_excel('AW_Sales_task.xlsx')

In [3]:
pd.to_datetime(sales['OrderDate'])

0       2011-05-31
1       2011-05-31
2       2011-05-31
3       2011-05-31
4       2011-05-31
           ...    
23598   2014-05-01
23599   2014-05-01
23600   2014-05-01
23601   2014-05-01
23602   2014-05-01
Name: OrderDate, Length: 23603, dtype: datetime64[ns]

In [4]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23603 entries, 0 to 23602
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   OrderDetailID       23603 non-null  int64         
 1   OrderDate           23603 non-null  datetime64[ns]
 2   DueDate             23603 non-null  datetime64[ns]
 3   ShipDate            23603 non-null  datetime64[ns]
 4   StatusID            23603 non-null  int64         
 5   Status              23603 non-null  object        
 6   CustomerID          23603 non-null  int64         
 7   SalesPersonID       23603 non-null  int64         
 8   TerritoryID         23603 non-null  int64         
 9   Territory           23603 non-null  object        
 10  TerritoryGroup      23603 non-null  object        
 11  ShipMethodID        23603 non-null  int64         
 12  ShipMethod          23603 non-null  object        
 13  ProductID           23603 non-null  int64     

In [5]:
sales.head()

,OrderDetailID,OrderDate,DueDate,ShipDate,StatusID,Status,CustomerID,SalesPersonID,TerritoryID,Territory,...,ShipMethod,ProductID,Product,ProductSubCategory,ProductCategory,OrderQty,UnitPrice,LineTotal,TaxAmt,TotalDue
0,15,2011-05-31,2011-06-12,2011-06-07,5,Shipped,29734,282,6,Canada,...,CARGO TRANSPORT 5,745,"HL Mountain Frame - Black, 48",Mountain Frames,Components,1,809.7600,809.7600,78.034563,912.180364
1,16,2011-05-31,2011-06-12,2011-06-07,5,Shipped,29734,282,6,Canada,...,CARGO TRANSPORT 5,743,"HL Mountain Frame - Black, 42",Mountain Frames,Components,1,714.7043,714.7043,68.874281,805.101794
2,17,2011-05-31,2011-06-12,2011-06-07,5,Shipped,29734,282,6,Canada,...,CARGO TRANSPORT 5,747,"HL Mountain Frame - Black, 38",Mountain Frames,Components,2,714.7043,1429.4086,137.748563,1610.203589
3,18,2011-05-31,2011-06-12,2011-06-07,5,Shipped,29734,282,6,Canada,...,CARGO TRANSPORT 5,712,AWC Logo Cap,Caps,Clothing,4,5.1865,20.7460,1.999241,23.370004
4,19,2011-05-31,2011-06-12,2011-06-07,5,Shipped,29734,282,6,Canada,...,CARGO TRANSPORT 5,715,"Long-Sleeve Logo Jersey, L",Jerseys,Clothing,4,28.8404,115.3616,11.117111,129.952808


In [6]:
sales.columns

Index(['OrderDetailID', 'OrderDate', 'DueDate', 'ShipDate', 'StatusID',
       'Status', 'CustomerID', 'SalesPersonID', 'TerritoryID', 'Territory',
       'TerritoryGroup', 'ShipMethodID', 'ShipMethod', 'ProductID', 'Product',
       'ProductSubCategory', 'ProductCategory', 'OrderQty', 'UnitPrice',
       'LineTotal', 'TaxAmt', 'TotalDue'],
      dtype='object')

In [7]:
#rename columns for match to exercise :
sales['Province'] = sales['Territory']
sales['ProvinceID'] = sales['TerritoryID']
sales['AgentID'] = sales['SalesPersonID']
sales['Agency'] = sales['TerritoryGroup']
sales['CustomerName'] = sales['CustomerID']

In [8]:
sales.to_csv('sales_1')

In [9]:
discount_codes = ['DISC10', 'DISC20', 'NONE', None]

In [10]:
def hash_customer_id(customer_id):
    return int(hashlib.sha256(str(customer_id).encode('utf-8')).hexdigest(), 16)

In [11]:
def generate_customer_name(customer_id):
    # Lists of potential first and last names
    first_names = ['John', 'Jane', 'Chris', 'Kathy', 'Mike', 'Nancy', 'Alex', 'Mary', 'Steve', 'Linda']
    last_names = ['Smith', 'Johnson', 'Williams', 'Jones', 'Brown', 'Davis', 'Miller', 'Wilson', 'Moore', 'Taylor']
    
    # Seed the random number generator with a hash of the customer ID
    seed = hash_customer_id(customer_id)
    random.seed(seed)
    
    # Generate a random first name and last name
    first_name = random.choice(first_names)
    last_name = random.choice(last_names)
    
    # Return the combined name
    return f"{first_name} {last_name}"

In [12]:
unique_customer_ids = sales['CustomerID'].unique()

# Generate a name for each unique customer ID
customer_names = {customer_id: generate_customer_name(customer_id) for customer_id in unique_customer_ids}

# Map the generated names back to the DataFrame
sales['CustomerName'] = sales['CustomerID'].map(customer_names)

In [13]:
def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
    finally:
        if conn:
            conn.close()

def execute_sql(conn, sql, data=None):
    """ Execute SQL statement """
    try:
        c = conn.cursor()
        if data is not None:
            c.execute(sql, data)
        else:
            c.execute(sql)
        conn.commit()
        
    except Error as e:
        print(e)

if __name__ == '__main__':
    database = "strasport_2.db"
    
    sql_create_globalTabel = """CREATE TABLE IF NOT EXISTS SalesData (
    ID INTEGER PRIMARY KEY,
    OrderDetailID INTEGER,
    OrderDate TEXT,
    DueDate TEXT,
    ShipDate TEXT,
    StatusID INTEGER,
    Status TEXT,
    CustomerID INTEGER,
    SalesPersonID INTEGER,
    TerritoryID INTEGER,
    Territory TEXT,
    TerritoryGroup TEXT,
    ShipMethodID INTEGER,
    ShipMethod TEXT,
    ProductID INTEGER,
    Product TEXT,
    ProductSubCategory TEXT,
    ProductCategory TEXT,
    OrderQty INTEGER,
    UnitPrice REAL,
    LineTotal REAL,
    TaxAmt REAL,
    TotalDue REAL,
    Province TEXT,
    ProvinceID INTEGER,
    AgentID INTEGER,
    Agency TEXT,
    CustomerName TEXT
);"""

    sql_create_store_table = """ CREATE TABLE IF NOT EXISTS Store (
                                        TerritoryID INTEGER PRIMARY KEY,
                                        Territory TEXT NOT NULL,
                                        TerritoryGroup TEXT NOT NULL
                                    ); """

    sql_create_client_table = """CREATE TABLE IF NOT EXISTS Client (
                                    ClientID INTEGER PRIMARY KEY,
                                    ClientName TEXT NOT NULL
                                );"""

    sql_create_product_table = """CREATE TABLE IF NOT EXISTS Product (
                                    ProductID INTEGER PRIMARY KEY,
                                    ProductType TEXT NOT NULL
                                );"""

    sql_create_transactions_table = """ CREATE TABLE IF NOT EXISTS Transactions (
                                        TransactionID INTEGER PRIMARY KEY,
                                        StoreID INTEGER NOT NULL,
                                        ClientID INTEGER NOT NULL,
                                        ProductID INTEGER NOT NULL,
                                        TransactionDate INTEGER,
                                        QuantitySold INTEGER NOT NULL,
                                        DiscountCode TEXT,
                                        FOREIGN KEY (StoreID) REFERENCES Store(TerritoryID),
                                        FOREIGN KEY (ClientID) REFERENCES Client (ClientID),
                                        FOREIGN KEY (ProductID) REFERENCES Product (ProductID),
                                        FOREIGN KEY (TransactionDate) REFERENCES DimTime(DateKey)

                                    ); """
    
    sql_create_DimensionTable = """ CREATE TABLE DimTime (
                                    DateKey INTEGER PRIMARY KEY,
                                    Day INTEGER,
                                    Month INTEGER,
                                    Quarter INTEGER,
                                    Semester INTEGER,
                                    Year INTEGER
                                ); """
    

    sql_Fqct_trunover = """CREATE TABLE FactTurnover (
    TurnoverID INTEGER PRIMARY KEY,
    DateKey INTEGER,
    StoreID INTEGER,
    ClientID INTEGER,
    AmountBilled DECIMAL,
    QuantityInvoiced INTEGER,
    DiscountValue DECIMAL,
    FOREIGN KEY (DateKey) REFERENCES DimTime(DateKey),
    FOREIGN KEY (StoreID) REFERENCES Store(TerritoryID),
    FOREIGN KEY (ClientID) REFERENCES Client ( ClientID)
);"""

    
    conn = sqlite3.connect(database)

   
    if conn is not None:
        execute_sql(conn, sql_create_store_table)
        execute_sql(conn, sql_create_client_table)
        execute_sql(conn, sql_create_product_table)
        execute_sql(conn, sql_create_transactions_table)
        execute_sql(conn, sql_create_DimensionTable)
        execute_sql(conn, sql_Fqct_trunover)
        
    else:
        print("Error! cannot create the database connection.")


In [14]:
#insert all the data in raw inside the database, but need to chunk it becasue too much lines
def insert_in_chunks(dataframe, chunk_size=100):
    for start_row in range(0, dataframe.shape[0], chunk_size):
        end_row = start_row + chunk_size
        chunk = dataframe[start_row:end_row]
        chunk.to_sql('SalesData', conn, if_exists='append', index=False, method=None)


insert_in_chunks(sales, 100)

In [15]:
#create def for insert data inside the table 
def insert_stores(conn, store):
    
    sql = ''' INSERT INTO Store(TerritoryID, Territory, TerritoryGroup)
              VALUES(?,?,?) '''
    execute_sql(conn, sql, store)
    

def insert_client(conn, client):
   
    sql = ''' INSERT INTO Client(ClientID, ClientName)
              VALUES(?,?) '''
    execute_sql(conn, sql, client)

def insert_product(conn, product):
    
    sql = ''' INSERT INTO Product(ProductID, ProductType)
              VALUES(?,?) '''
    execute_sql(conn, sql, product)

def insert_transaction(conn, transaction):
    
    sql = ''' INSERT INTO Transactions(TransactionID, StoreID, ClientID, ProductID, TransactionDate, QuantitySold, DiscountCode)
              VALUES(?,?,?,?,?,?,?) '''
    execute_sql(conn, sql, transaction)


In [16]:
unique_clients = sales[['CustomerID', 'CustomerName']].drop_duplicates()
for _, row in unique_clients.iterrows():
    insert_client(conn, (row['CustomerID'], row['CustomerName']))


In [17]:
for _, row in sales.iterrows():
    stores = (row['TerritoryID'], row['Territory'],row['TerritoryGroup'])
    insert_stores(conn, stores)

UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.

UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.

UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.

UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.

UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.

UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.TerritoryID
UNIQUE constraint failed: Store.

In [18]:
unique_products = sales[['ProductID', 'ProductSubCategory']].drop_duplicates()
for _, row in unique_products.iterrows():
    insert_product(conn, (row['ProductID'], row['ProductSubCategory']))


In [19]:
sales['OrderDate'] = pd.to_datetime(sales['OrderDate'])

for _, row in sales.iterrows():
   
    order_date_str = row['OrderDate'].strftime('%Y-%m-%d %H:%M:%S')
    
    transaction = (
        row['OrderDetailID'],  
        row['AgentID'],       
        row['CustomerID'],
        row['ProductID'],
        order_date_str,       
        row['OrderQty'],       
        row.get('DiscountCode', None) 
    )
    insert_transaction(conn, transaction)


In [20]:
sales.columns

Index(['OrderDetailID', 'OrderDate', 'DueDate', 'ShipDate', 'StatusID',
       'Status', 'CustomerID', 'SalesPersonID', 'TerritoryID', 'Territory',
       'TerritoryGroup', 'ShipMethodID', 'ShipMethod', 'ProductID', 'Product',
       'ProductSubCategory', 'ProductCategory', 'OrderQty', 'UnitPrice',
       'LineTotal', 'TaxAmt', 'TotalDue', 'Province', 'ProvinceID', 'AgentID',
       'Agency', 'CustomerName'],
      dtype='object')

In [21]:
if conn:
    conn.close()